<h1>Predicting Crew Members with Linear Regression using PySpark</h1>

In this notebook, I am creating a regression model that will help predict how many crew members will be needed for Hyundai Heavy Industries ships.

<h2>Step 1: Create a new Spark session</h2>

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PredictingCrewMembers").getOrCreate()

<h2>Step 2: Load the data</h2>

First I'm loading a csv with cruise ships information called <code>cruise_ship_info.csv</code>, which you can find online <a href="https://raw.githubusercontent.com/kushangbhatt/CrewMemberPrediction/master/cruise_ship_info.csv">here</a>.

In [ ]:
file = "cruise_ship_info.csv"
df = spark.read.csv(file, header = True, inferSchema = True)

In [ ]:
print("This dataset has {} columns.\n\nThe names of these columns are:\n  {}".format(len(df.columns), df.columns))

## Exploring data

Now, it is important to first examine the data we have. The first five rows in the dataset look like this:

In [ ]:
df.show(5)

Also, we can see the schema of the database in a tree format:

In [ ]:
df.printSchema()

The target variable is **crew**, which is continuous. So are most of the features in this dataset. Only **Ship_name** and **Cruise_line** are categorical variables.

Below, a summary of the database is shown in <em>Pandas</em> format. Here we can see mean, standard deviation, minimum and maximum values for each numerical column. Also, notice that there are 158 rows in this dataset.

In [ ]:
df.describe().toPandas()

It is time to explore how many unique values are there in both categorical variables:

In [ ]:
for column in ["Ship_name", "Cruise_line"]:    
    print("There are %s different values in %s column." % (df.select(column).distinct().count(), column))

**Ship_name** has roughly as many distinct values as rows are there in the dataset. Then, I think this feature is not very informative for predicting crew members, so I decide not to include it in the linear regression model.

Next, I am converting the **Cruise_line** column to numerical labels for each string. I want to include this variable in the linear regression model.

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Cruise_line", outputCol="line_id")
indexed = indexer.fit(df).transform(df)
indexed.select("line_id").describe().show()

We see that the strings in **Cruise_line** have been transformed to numerical values between 0 and 19. Now, we can include this numerical variable in the linear regression model.

In [ ]:
print([int(row.line_id) for row in indexed.select("line_id").distinct().orderBy("line_id").collect()])

In [ ]:
for i in range(len(indexed.schema.names)):
    print("Column number", i, "is", indexed.schema.names[i])

We want to include in the linear regression model as predictor variables all columns between 2 and 9 except column number 8, which is the target variable. Knowing this, the vector assembler with all features for the linear regression model is created in the next block of code.

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = [indexed.schema.names[i] for i in [9] + list(range(2,8))],
                            outputCol = "features")
output = assembler.transform(indexed)
output.select("features").show(5)

Features vector has been correctly generated. Next, I am fitting a linear regression model that considers these seven features.

In [ ]:
from pyspark.ml.regression import LinearRegression

# Splitting of final_data in train (70%) and test (30%) sets. Random seed is
#   set to today's date, 2020-12-21

train_data, test_data = output.randomSplit([0.7, 0.3], seed = 20201221)


# Fitting linear regression model with train_data

lr = LinearRegression(labelCol="crew")
lr_model = lr.fit(train_data)

How well does this model predicts crew members? I am evaluating its performance in terms of RMSE and R²:

In [ ]:
# Evaluating linear regression model with test_data
test_results = lr_model.evaluate(test_data)


# Outputting R-squared of this model
print("RMSE of model =", test_results.rootMeanSquaredError)
print("R-squared of model =", test_results.r2)

R² is very close to 1 and RMSE is quite low, so this model seems adequate to make predictions about the number of crew members given some features.

However, what would had happened if I hadn't included **Cruise_line** as a numerical variable (**line_id**) in the model? Let's see:

In [ ]:
assembler2 = VectorAssembler(inputCols = [df.columns[i] for i in range(2,8)], outputCol = "features")

output2 = assembler2.transform(df)

# Splitting of final_data in train (70%) and test (30%) sets. Random seed is
#   set to today's date, 2020-11-04

train_data2, test_data2 = output2.randomSplit([0.7, 0.3], seed = 20201221)


# Fitting linear regression model with train_data

lr_model2 = lr.fit(train_data2)


# Evaluating linear regression model with test_data
test_results2 = lr_model2.evaluate(test_data2)


# Outputting R-squared of this model
print("RMSE of model =", test_results2.rootMeanSquaredError)
print("R-squared of model =", test_results2.r2)

We can see that both models perform very well, in terms of RMSE and R². Then, we might think that **Cruise_line** is not a very informative predictor variable.

This is just a very simple example, but linear regression can be applied to a lot of more complex and interesting tasks using Spark. I hope you have enjoyed this notebook, see you soon!